## CS431/631 Data Intensive Distributed Analytics
### Fall 2019 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:**
* **ID:** 

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [2]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [3]:
import random

num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.141718


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

The pi-estimate computation is based on heuristic: by defination π is the area A(Circle) of a circle with radius r=1. A square with the area 4 circumscribes the circle. the ration of the 2 areas  equal π/4. If n random points are picked inside the square, then n(in) points will be inside the circle and n(out) points will be outside the circle. Thus n(in) + n(out) = n. And the probability of point lying inside the circle is n(in)/n. 
Thus,
π/4 = n(in)/n --> π = 4*n(in)/n

Spark creates a RDD and divides the data, which conatins n consecutive numbers from 0 to 100000000. Then it filter outs the points from sample that lie inside the unit circle. Finally, the count of the points are taken and stored in the count variable. Using the above formula the value of π is calculated. In this case Transfrmation is the filter function and the Action is the count function.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [4]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    # your solution to Question 2 here
    # reading the input file
    text = sc.textFile("Shakespeare.txt")
    #Tokenizing the text and using Distinct Transformation to get the distinct words and later counting the words.
    tokens_count = text.flatMap(simple_tokenize).distinct().count()
    return tokens_count
    
print("Number of distinct tokens = ", count_distinct_tokens())

Number of distinct tokens =  25975


---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [5]:
from simple_tokenize import simple_tokenize
import itertools as ite


# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    # your solution to Question 3 here
    #reading the input file
    text_pairs = sc.textFile("Shakespeare.txt")
    #Tokenizing the words and using FlatMap Transformation and Disticnt Transformation to create distinct pairs
    #and then taking the count
    pairs = text_pairs.flatMap(lambda line : ite.permutations(set(simple_tokenize(line)),2)).distinct().count()
    return pairs
print("Number of distinct pairs = ", count_distinct_pairs())
    

Number of distinct pairs =  1969760


---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [6]:
from simple_tokenize import simple_tokenize
from collections import Counter

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    # your solution to Question 4 here    
    #reading the input
    text_prob = sc.textFile("Shakespeare.txt")
    #Counting number of lines in the input file
    num_lines = text_prob.count()
    #Tokenizing the sentences
    tokens_line = text_prob.flatMap(lambda line: set(simple_tokenize(line)))
    #Calculating the count of unique tokens per line
    count_words = tokens_line.map(lambda word : (word,1)).reduceByKey(lambda x,y : x+y)
    #Interchanging the key, value and sorting the words based on the count
    words_sorted = count_words.map(lambda x : (x[1],x[0])).sortByKey(False)
    #Calculating the probability and returning the top 50 probability
    prob_count_token = words_sorted.map(lambda x : (x[0]/num_lines, x[0],x[1])).take(50)
    return prob_count_token

print("The top 50 distinct token probabilites are : ",top_50_tokens_probabilities())

The top 50 distinct token probabilites are :  [(0.2009178657172255, 24604, 'and'), (0.19843538192686472, 24300, 'the'), (0.1523542765682928, 18657, 'i'), (0.148924529226347, 18237, 'to'), (0.1357526662202551, 16624, 'of'), (0.10844534452628657, 13280, 'a'), (0.09959332995802643, 12196, 'you'), (0.09430988583840991, 11549, 'my'), (0.08667461497003054, 10614, 'in'), (0.08630714204053634, 10569, 'that'), (0.07150206601447026, 8756, 'is'), (0.06720671577193814, 8230, 'not'), (0.06167012363422561, 7552, 'with'), (0.06039621747864574, 7396, 'me'), (0.05982459292165477, 7326, 'for'), (0.05836286726877787, 7147, 'it'), (0.054402325695340446, 6662, 'be'), (0.05246696826667102, 6425, 'this'), (0.05228731483447386, 6403, 'his'), (0.050899083767495794, 6233, 'your'), (0.05067043394469941, 6205, 'but'), (0.0474938346208496, 5816, 'he'), (0.046889545803459144, 5742, 'have'), (0.04132028940534714, 5060, 'thou'), (0.04015254209606559, 4917, 'as'), (0.03952375508337552, 4840, 'him'), (0.039491090822976

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [12]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # your solution to Question 5 here
    #reading the input
    text_pmi = sc.textFile("Shakespeare.txt")
    #counting the number of lines
    no_lines = text_pmi.count()
    #Counting distinct tokens
    tokens_cnt = text_pmi.flatMap(lambda line : set(simple_tokenize(line))).map(lambda w: (w,1)).reduceByKey(lambda x,y: x+y)
    #storing the cooccurence pair in a RDD
    cooccurence_rdd = text_pmi.flatMap(lambda line: ite.permutations(set(simple_tokenize(line)),2))
    #Counting the cooccurence pair
    cooccurence_count = cooccurence_rdd.map(lambda pair:(pair,1)).reduceByKey(lambda x,y : x+y).map(lambda x: (x[0][0],(x[0][1],x[1])))
    #combining tokens in the pair with their corresponding values
    combined_count_1 = cooccurence_count.join(tokens_cnt).map(lambda x:(x[1][0][0],(x[0],x[1][0][1],x[1][1])))
    combined_count_2 = combined_count_1.join(tokens_cnt).map(lambda x:((x[0],x[1][0][0]),x[1][0][1],x[1][1],x[1][0][2]))
    #Filtering the values based on the Threshold
    filter_rdd = combined_count_2.filter(lambda x: x[1] > threshold)
    #Calculating the PMI values
    pmi_rdd = filter_rdd.map(lambda x: (x[0],cal_pmi(x[1],x[2],x[3],no_lines),x[1],x[2],x[3]))
    return pmi_rdd.collect()

def cal_pmi(p,t1,t2,n):
    pmi = log((p*n/(t1*t2)),10)
    return pmi
#calling the PMI function and passing the threshold value:
pmi_val = PMI(10)
pmi_val

[(('we', 'but'), 0.15257787280013135, 225, 3125, 6205),
 (('we', 'where'), 0.30131014328253264, 62, 3125, 1214),
 (('we', 'now'), 0.18323590254754024, 106, 3125, 2724),
 (('we', 'only'), 0.18235990714630768, 12, 3125, 309),
 (('we', 'in'), 0.07104874028733191, 319, 3125, 10614),
 (('we', 'when'), 0.36011968835480945, 119, 3125, 2035),
 (('we', 'field'), 0.3912850193234148, 12, 3125, 191),
 (('we', 'of'), -0.08105572621145361, 352, 3125, 16624),
 (('we', 'days'), 0.42120184123899396, 14, 3125, 208),
 (('we', 'say'), 0.126815949894289, 57, 3125, 1668),
 (('we', 'an'), -0.1577086078789784, 32, 3125, 1803),
 (('we', 'tell'), -0.08810409685206964, 22, 3125, 1056),
 (('we', 'he'), -0.2964259939357638, 75, 3125, 5816),
 (('we', 'why'), 0.18794361072483434, 57, 3125, 1449),
 (('we', 'are'), 0.6326571394040973, 361, 3125, 3296),
 (('we', 'give'), 0.06329892993548952, 39, 3125, 1321),
 (('we', 'yet'), 0.20535813648821777, 64, 3125, 1563),
 (('we', 'leave'), 0.10311173845706754, 21, 3125, 649),
 

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [16]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here
    #reading the input
    text_pmi = sc.textFile("Shakespeare.txt")
    #counting the number of lines
    no_lines = text_pmi.count()
    #Counting distinct tokens
    tokens_cnt = text_pmi.flatMap(lambda line : set(simple_tokenize(line))).map(lambda w: (w,1)).reduceByKey(lambda x,y: x+y)
    single_count = text_pmi.flatMap(lambda line : set(simple_tokenize(line))).map(lambda w: (w,1)).reduceByKey(lambda x,y: x+y).map(lambda x: (x[1],x[0])) 
    #storing the cooccurence pair in a RDD
    cooccurence_rdd = text_pmi.flatMap(lambda line: ite.permutations(set(simple_tokenize(line)),2))
    #Counting the cooccurence pair
    cooccurence_count = cooccurence_rdd.map(lambda pair:(pair,1)).reduceByKey(lambda x,y : x+y).map(lambda x: (x[0][0],(x[0][1],x[1])))
    #combining tokens in the pair with their corresponding values
    combined_count_1 = cooccurence_count.join(tokens_cnt).map(lambda x:(x[1][0][0],(x[0],x[1][0][1],x[1][1])))
    combined_count_2 = combined_count_1.join(tokens_cnt).map(lambda x:((x[0],x[1][0][0]),x[1][0][1],x[1][1],x[1][0][2]))
    #Filtering the values based on the Threshold
    filter_rdd = combined_count_2.filter(lambda x: x[1] > threshold)
    #Calculating the PMI values
    pmi_rdd = filter_rdd.map(lambda x: (x[0],cal_pmi(x[1],x[2],x[3],no_lines),x[1],x[2],x[3]))
    rearranged_rdd = pmi_rdd.map(lambda x: (x[3],(x[0],x[1],x[2],x[4])))
    
    #Joining the RDD 2 get the desired result
    dis_rdd = rearranged_rdd.join(single_count).map(lambda x:(x[1][1],(x[1][0][0],x[1][0][1],x[1][0][2],x[0],x[1][0][3])))
    #selecting N random pairs
    one_token_list = dis_rdd.groupByKey().take(samp_size)
    return one_token_list


def cal_pmi(p,t1,t2,n):
    pmi = log((p*n/(t1*t2)),10)
    return pmi


one_token = PMI_one_token(1000,5)
if one_token == []:
    print("No token pair for the given threshold")
else:
    for tk in one_token:
        print("(",tk[0],",",[t for t in tk[1]],")")
    

( of , [(('of', 'in'), 0.05276010163400139, 1627, 16624, 10614), (('of', 'i'), -0.08531809933104943, 2081, 16624, 18657), (('of', 'the'), 0.34294075191889306, 7266, 16624, 24300), (('of', 'that'), 0.03856379243802408, 1568, 16624, 10569), (('of', 'to'), -0.09813466615111513, 1975, 16624, 18237), (('of', 'a'), 0.13551796879761388, 2463, 16624, 13280), (('of', 'and'), 0.028305447826683594, 3565, 16624, 24604), (('of', 'you'), -0.06090498976622853, 1439, 16624, 12196), (('of', 'his'), 0.07998442061341456, 1045, 16624, 6403), (('of', 'this'), 0.15174819961629782, 1237, 16624, 6425), (('of', 'is'), 0.020115230526370407, 1245, 16624, 8756), (('of', 'my'), 0.011802791063262602, 1611, 16624, 11549)] )
( are , [(('are', 'you'), 0.5107241714442197, 1064, 3296, 12196)] )
( he , [(('he', 'the'), 0.0010898497060675269, 1157, 5816, 24300), (('he', 'and'), 0.030654582495617602, 1254, 5816, 24604), (('he', 'is'), 0.4030727512744563, 1052, 5816, 8756)] )
( i , [(('i', 'but'), 0.16991073624533976, 1398,

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.